In [259]:
from googleplaces import GooglePlaces, types, lang
from googlemaps import Client as GoogleMaps
from gmplot import gmplot
import pickle
import time
from decimal import Decimal
import pandas as pd
# pip install gmplot
import os

near_By_Roads = []


In [154]:
def Read_API_KEY_From_File(file_Path):
    list_Key = []
    with open(file_Path) as file_text:
        for key in file_text:
            list_Key.append(key.strip())
    return list_Key

api_key_File_Path = r'D:\Product\API_KEY\API_KEY.txt'

list_Key = Read_API_KEY_From_File(api_key_File_Path)

PLACE_API_KEY = list_Key[0]
ROADS_API_KEY = list_Key[1]

file_Path_To_Store_Data = 'Delhi_Proper_Places.pkl'


google_places = GooglePlaces(PLACE_API_KEY)
gmaps = GoogleMaps(ROADS_API_KEY)

In [194]:
list_Places_Delhi = []
file_Path_To_Store_Data = 'Delhi_Proper_Places.pkl'

with open(file_Path_To_Store_Data, 'rb') as f:
    list_Places_Delhi = pickle.load(f)
    
len(list_Places_Delhi)

550

In [155]:
all_Types_To_Search = [types.TYPE_PARKING, 
                       types.TYPE_TRAIN_STATION,
                       types.TYPE_TAXI_STAND, 
                       types.TYPE_GAS_STATION,
                       
                       types.TYPE_MOVIE_THEATER,
                       types.TYPE_AIRPORT, 
                       types.TYPE_SHOPPING_MALL
                      ]

In [235]:
list_Known_Places = []
list_Known_Places.append((28.578999, 77.196258, 'Swami Vivekanand Park'))
# list_Known_Places.append((28.6127, 77.2773, 'Akshardham'))
# list_Known_Places.append((28.6365, 77.1831, 'New Rajinder Nagar Post Office'))
# list_Known_Places.append((28.5244, 77.1855, 'Qutub Minar'))
list_Known_Places.append((28.5681, 77.2058, 'Safdarjung Hospital'))
# list_Known_Places.append((28.5997, 77.0990, 'Indira Park')) 
# list_Known_Places.append((28.6390, 77.0745, 'PVR Vikaspuri')) 

list_Known_Places = []
place_IDs = []

max_Places_Required = 100

for index in range (0, len(list_Places_Delhi), 3):
     
    place = list_Places_Delhi[index]
    
    result = google_places.nearby_search(lat_lng=place.geo_location, radius=500, type= types.TYPE_ROUTE)
    
    result = result.places
    
    if len(result) > 0:
        place = result[0]
        if (place.place_id not in place_IDs):
            list_Known_Places.append(((place.geo_location['lat']), (place.geo_location['lng']), place.name))
            place_IDs.append(place.place_id)
        time.sleep(1)
        
    if len(list_Known_Places) == max_Places_Required:
        index = len(list_Places_Delhi)
        break;
        
    print ('Status: ' + str(index) + '  ' + 'Total fecthed : ' + str(len(list_Known_Places)) + '/' + str(max_Places_Required) )

    
print(len(list_Known_Places))
    
    


Status: 0  Total fecthed : 1/100
Status: 3  Total fecthed : 2/100
Status: 6  Total fecthed : 3/100
Status: 9  Total fecthed : 4/100
Status: 12  Total fecthed : 5/100
Status: 15  Total fecthed : 6/100
Status: 18  Total fecthed : 7/100
Status: 21  Total fecthed : 8/100
Status: 24  Total fecthed : 9/100
Status: 27  Total fecthed : 10/100
Status: 30  Total fecthed : 11/100
Status: 33  Total fecthed : 12/100
Status: 36  Total fecthed : 13/100
Status: 39  Total fecthed : 14/100
Status: 42  Total fecthed : 15/100
Status: 45  Total fecthed : 16/100
Status: 48  Total fecthed : 17/100
Status: 51  Total fecthed : 18/100
Status: 54  Total fecthed : 19/100
Status: 57  Total fecthed : 20/100
Status: 60  Total fecthed : 21/100
Status: 63  Total fecthed : 22/100
Status: 66  Total fecthed : 23/100
Status: 69  Total fecthed : 24/100
Status: 72  Total fecthed : 25/100
Status: 75  Total fecthed : 26/100
Status: 78  Total fecthed : 27/100
Status: 81  Total fecthed : 28/100
Status: 84  Total fecthed : 29/10

In [249]:
# delhi_Data_Columns = [
#                          'Place_Name',
#                          'Latitude', 'Longitude',    
#                          'No_Of_Nearest_PARKING', 
#                          'No_Of_Nearest_TRAIN_STATION', 
#                          'No_Of_Nearest_TAXI_STAND', 
#                          'No_Of_Nearest_GAS_STATION', 
#                          'No_Of_Nearest_MOVIE_THEATER', 
#                          'No_Of_Nearest_AIRPORT', 
#                          'No_Of_Nearest_SHOPPING_MALL'
#                     ]

place_Columns = [
                         'Place_Name',
                         'Latitude', 'Longitude']

delhi_Data_Columns = place_Columns + all_Types_To_Search

delhi_Data_Columns


['Place_Name',
 'Latitude',
 'Longitude',
 'parking',
 'train_station',
 'taxi_stand',
 'gas_station',
 'movie_theater',
 'airport',
 'shopping_mall']

In [250]:
df = pd.DataFrame( columns = delhi_Data_Columns)

In [251]:
df.head(1)

,Place_Name,Latitude,Longitude,parking,train_station,taxi_stand,gas_station,movie_theater,airport,shopping_mall


In [252]:
for noted_places in list_Known_Places:
    new_Row = []
    new_Row.append(noted_places[2])
    new_Row.append(noted_places[0])
    new_Row.append(noted_places[1])
    
    for index in range(3, len(df.columns)):
        new_Row.append(0)
        
#     df.loc[len(df)] = new_Row

In [253]:
def Get_Query_Result(location_To_Search, max_Radius, types_To_Search):
    queries = []
    
    query_result =  google_places.nearby_search(lat_lng=location_To_Search, radius=max_Radius, type= types_To_Search)
    queries.append(query_result)
    
    while query_result.has_next_page_token:
        print('Next Tokens')
        time.sleep(1)
        query_result = google_places.nearby_search( pagetoken=query_result.next_page_token)  
        queries.append(query_result)

            
    return queries

In [257]:
def Add_Data():
    index = 0
    
    max_Radius = 1000 # meteres
    
    for noted_places in list_Known_Places:     
        new_Row = []
        new_Row.append(noted_places[2])
        new_Row.append(noted_places[0])
        new_Row.append(noted_places[1])
        
        location_To_Search = {'lat' : noted_places[0], 'lng' : noted_places[1]}    
        
        for column_Name in df.columns:
            if column_Name not in place_Columns:
#                 print('---------------------------------------------', column_Name)
                result = Get_Query_Result(location_To_Search, max_Radius, column_Name)
                for i in range (len(result)):
                    count = 0
                    for place in result[i].places:
                        count = count + 1
                        
                new_Row.append (count)
        time.sleep(1)
        
        df.loc[len(df)] = new_Row
        index = index + 1
        
        print(index)
                   

In [272]:
# Add_Data()
df.head(100)

df.to_csv('Delhi_Scoring_Data.csv', index = False)

print(len(df))

df.drop_duplicates(inplace = True)

df.reset_index(drop = True, inplace = True)

100


In [271]:

location_To_Search = {'lat' : 28.590583, 'lng' : 77.231241}

result = Get_Query_Result(location_To_Search, 1000, types.TYPE_ROUTE)




In [189]:
for i in range (len(result)):
    for place in result[i].places:
        print (place)
        print (place.name)
        print (place.get_details)
        print (place.place_id)

<Place name="Lodhi Road", lat=28.5916382, lng=77.231126>
Lodhi Road
<bound method Place.get_details of <Place name="Lodhi Road", lat=28.5916382, lng=77.231126>>
ChIJ2_g3PeXiDDkRP7nuDB4Ei1I


In [192]:
def Get_Nearest_Road_Details(location_To_Search):
    result = gmaps.nearest_roads(location_To_Search)
    
    if (len(result) > 0):
        placeId = str(result[0]['placeId'])
        place_Object = google_places.get_place(placeId)
        return (place_Object)
    else:
        return False
    
location_To_Search = {'lat' : 28.5916382, 'lng' : 77.231126}

place =  Get_Nearest_Road_Details(location_To_Search)

place

<Place name="Lodhi Road", lat=28.5916382, lng=77.231126>

550